<a href="https://www.kaggle.com/code/omarjfr/tf-idf?scriptVersionId=168551877" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
!pip install transformers

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline
import numpy as np
import pandas as pd
import scipy.sparse as sp
from collections import Counter
from numpy.linalg import norm
import nltk
import re
import spacy
from nltk.corpus import stopwords
nlp = spacy.load("en_core_web_sm")
import re
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup

In [3]:
def generate_documents_in_fields(input_phrases, max_length=200):

    text_generator = pipeline("text-generation", model='gpt2')# give a pipline a task

    # Generate documents for each input phrase
    documents_per_field = {}
    for field, phrase in input_phrases.items():
        documents = []
        generated_text = text_generator(phrase, max_length=max_length, num_return_sequences=1,truncation=True )[0]['generated_text']
        documents.append(generated_text)
        documents_per_field[field] = documents
    return documents_per_field

In [4]:
input_phrases = {
    "Technology": "The future of technology",
    "Healthcare": "The impact of AI in healthcare",
    "Finance": "The future of finance with blockchain technology"
}
generated_documents = generate_documents_in_fields(input_phrases)


for field, documents in generated_documents.items():
    print(f"Field: {field}")
    for i, document in enumerate(documents, start=1):
        print(f"Document {i}:")
        print(document)
        print()


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Field: Technology
Document 1:
The future of technology can have some very serious ramifications. This is especially true for emerging digital technologies, like Google's Pixel. They are now being used not only as the operating system for smartphones, but for other services that support Google Play services, such as Maps, YouTube, and even a variety of Android-powered apps. For example, one study by the University of Michigan University, for both the Pew Research Center and the Center for Research on Education, found that only 2 percent of teachers polled said they are likely to support a wide variety of technology changes, and only 11 percent of teachers said they would support changing the standards or setting the course for any given class. These attitudes were similar between 2010 and 2013.

The future of mobile Internet adoption has been discussed in detail in an important paper by Paul J. Zierz and David A. Levine in the journal Globalization and Social Networks, which looked at t

In [5]:
generated_documents

{'Technology': ["The future of technology can have some very serious ramifications. This is especially true for emerging digital technologies, like Google's Pixel. They are now being used not only as the operating system for smartphones, but for other services that support Google Play services, such as Maps, YouTube, and even a variety of Android-powered apps. For example, one study by the University of Michigan University, for both the Pew Research Center and the Center for Research on Education, found that only 2 percent of teachers polled said they are likely to support a wide variety of technology changes, and only 11 percent of teachers said they would support changing the standards or setting the course for any given class. These attitudes were similar between 2010 and 2013.\n\nThe future of mobile Internet adoption has been discussed in detail in an important paper by Paul J. Zierz and David A. Levine in the journal Globalization and Social Networks, which looked at the relation

In [6]:
doc_1=generated_documents.get("Technology") 
doc_2=generated_documents.get("Healthcare")
doc_3=generated_documents.get("Finance")

documents = [doc_1, doc_2, doc_3]

In [7]:
documents = ['\n'.join(doc) for doc in documents]
documents

["The future of technology can have some very serious ramifications. This is especially true for emerging digital technologies, like Google's Pixel. They are now being used not only as the operating system for smartphones, but for other services that support Google Play services, such as Maps, YouTube, and even a variety of Android-powered apps. For example, one study by the University of Michigan University, for both the Pew Research Center and the Center for Research on Education, found that only 2 percent of teachers polled said they are likely to support a wide variety of technology changes, and only 11 percent of teachers said they would support changing the standards or setting the course for any given class. These attitudes were similar between 2010 and 2013.\n\nThe future of mobile Internet adoption has been discussed in detail in an important paper by Paul J. Zierz and David A. Levine in the journal Globalization and Social Networks, which looked at the relationship between mo

## Clean Text

In [56]:
def clean_text(document):
    cleaned_documents = []
    cleaned_words = []
   # Process the text using spaCy
    doc = nlp(document)
    # Extract lemmatized tokens
    lemmatized_tokens = [token.lemma_ for token in doc]
    # Join the lemmatized tokens into a sentence
    cleaned_doc = ' '.join(lemmatized_tokens)
    cleaned_doc = cleaned_doc.lower()
    # Remove html tags
    cleaned_doc = BeautifulSoup(cleaned_doc, "html.parser").get_text()
    # Remove links and emails
    cleaned_doc = re.sub(r'http\S+|www\S+|[\w\.-]+@[\w\.-]+', '', cleaned_doc)
    # Remove any thing except words, numbers, and space
    cleaned_doc = re.sub(r'[^\w\s]', '', cleaned_doc)
    # Remove numbers
    cleaned_doc = re.sub(r'[0-9]', '', cleaned_doc)
    # Remove emojis and non ascii characters
    cleaned_doc = re.sub(r'[^\x00-\x7F]+', '', cleaned_doc)
    # Remove stop words
    stop_words = set(stopwords.words("english"))
    cleaned_doc = ' '.join([word for word in cleaned_doc.split() if word not in stop_words])
    # Split sentences to words
    cleaned_words.extend(cleaned_doc.split())
    cleaned_doc = ' '.join([word for word in cleaned_doc.split() if (len(word) > 1 or word == 'i')])
    cleaned_documents.append(cleaned_doc)
    return cleaned_documents, cleaned_words

In [57]:
clean_text,BagOfWords = zip(*[clean_text(doc) for doc in documents])

In [58]:
# Documents after clean
clean_text = [item for sublist in clean_text for item in sublist]
clean_text

['future technology serious ramification especially true emerge digital technology like google pixel use operating system smartphone service support google play service maps youtube even variety android power app example one study university michigan university pew research center center research education find percent teacher poll say likely support wide variety technology change percent teacher say would support change standard set course give class attitude similar future mobile internet adoption discuss detail important paper paul zierz david levine journal globalization social networks look relationship mobile internet use global economy author paper',
 'impact ai healthcare may much potential bias physician may actually due lack high availability good quality neurosurgeon advanced medical knowledge like neurosurgeon neurologist example research use mri result perform placebo control trial researcher blind single outcome measure eg patient age word use good quality neurosurgeon re

In [71]:
flat_words = [word for sublist in BagOfWords for word in sublist]

# Get unique words
unique_words = set(flat_words)

print("Unique Words:", unique_words)

Unique Words: {'paper', 'mobile', 'detail', 'mean', 'healthcare', 'term', 'service', 'quality', 'transaction', 'still', 'perform', 'j', 'neurosurgeon', 'center', 'germany', 'android', 'potential', 'researcher', 'support', 'youtube', 'economy', 'social', 'agree', 'neurosurgery', 'relationship', 'maps', 'example', 'like', 'sync', 'digital', 'internet', 'government', 'patient', 'zierz', 'placebo', 'necessarily', 'stick', 'distribute', 'would', 'key', 'ai', 'power', 'research', 'development', 'impact', 'come', 'trial', 'privacy', 'pew', 'time', 'likely', 'ramification', 'purpose', 'although', 'variety', 'advanced', 'begin', 'rate', 'interesting', 'networks', 'available', 'wide', 'health', 'globalization', 'app', 'affordable', 'make', 'basis', 'information', 'may', 'risk', 'allow', 'certainly', 'standard', 'knowledge', 'collect', 'banking', 'poll', 'party', 'david', 'look', 'future', 'study', 'find', 'author', 'eg', 'set', 'result', 'global', 'keep', 'finance', 'pixel', 'blockchain', 'nih',

## TF-IDF

In [59]:
tfidf_vectorizer = TfidfVectorizer()

#Transform documents to TF-IDF vectors
tfidf_vectors = tfidf_vectorizer.fit_transform(clean_text)


print("Shape of TF-IDF matrix:", tfidf_vectors.shape)

print("TF-IDF matrix:")
print(tfidf_vectors.toarray())

print("Vocabulary:")
print(tfidf_vectorizer.get_feature_names_out())

Shape of TF-IDF matrix: (3, 180)
TF-IDF matrix:
[[0.         0.         0.0697837  0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.09175718 0.         0.09175718 0.         0.         0.09175718
  0.09175718 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.18351436 0.         0.18351436 0.09175718 0.         0.
  0.         0.09175718 0.         0.09175718 0.         0.09175718
  0.         0.09175718 0.09175718 0.         0.         0.09175718
  0.09175718 0.         0.         0.09175718 0.         0.09175718
  0.0697837  0.0697837  0.         0.         0.09175718 0.
  0.1395674  0.         0.09175718 0.09175718 0.09175718 0.
  0.18351436 0.         0.         0.         0.         0.
  0.         0.0697837  0.         0.         0.         0.18351436
  0.09175718 0.         0.         0.         0.         0.
  0.09175718 0.0697837  0.0697837  0.         0.09175718 0.
  0.09175718

In [60]:
df = pd.DataFrame(tfidf_vectors.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
df.head()

,actually,add,adoption,advanced,affordable,age,agree,ai,allow,also,...,value,variety,verifiable,well,wide,word,world,would,youtube,zierz
0,0.000000,0.000000,0.069784,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.183514,0.000000,0.000000,0.091757,0.000000,0.000000,0.069784,0.091757,0.091757
1,0.093021,0.000000,0.000000,0.093021,0.000000,0.093021,0.000000,0.093021,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.093021,0.000000,0.093021,0.000000,0.070745,0.000000,0.000000
2,0.000000,0.065326,0.049682,0.000000,0.065326,0.000000,0.130652,0.000000,0.065326,0.065326,...,0.065326,0.000000,0.065326,0.000000,0.000000,0.000000,0.065326,0.000000,0.000000,0.000000


In [67]:
df["technology"]

0    0.209351
1    0.000000
2    0.298092
Name: technology, dtype: float64

# TF-IDF from scratch

In [89]:
def tf(clean_text, words):
    BagOfWords = [doc.split() for doc in clean_text]
    Unique_Words = list(set([word for sublist in BagOfWords for word in sublist]))
    features_dict = {w:0 for w in Unique_Words}
    tf = []
    for doc in clean_text:
        bowf_doc = Counter(doc.split())
        all_f = Counter(features_dict) # Counter object 
        bowf_doc.update(all_f)
        tf.append(bowf_doc)
    return pd.DataFrame(tf, columns=Unique_Words), Unique_Words
    
def df(tf):#-> n
    features_names = list(tf.columns)#words
    df = np.diff(sp.csc_matrix(tf, copy=True).indptr)#convert to sparce matrix and count of non-zero elements in each column.
    df = 1 + df #smoothing(prevent zero division errors).
    return df
    
def idf(df, clean_text):
    N = 1 + len(clean_text)# avoid log(0)->infinite
    idf = (1.0 + np.log(float(N) / df))       
    return idf

def tfidf(tf, idf):        
    tf = np.array(tf, dtype='float64')
    tfidf = tf * idf
    norms = norm(tfidf , axis=1)
    return (tfidf / norms[:,None])

In [90]:
tf_matrix, Unique_Words = tf(clean_text, BagOfWords)
idf_values= idf(df_values, clean_text)
tfidf_matrix = tfidf(tf_matrix, idf_values)

In [63]:
df = pd.DataFrame(tfidf_matrix, columns=Unique_Words)
df = df.sort_index(axis=1)
df.head()

,actually,add,adoption,advanced,affordable,age,agree,ai,allow,also,...,value,variety,verifiable,well,wide,word,world,would,youtube,zierz
0,0.000000,0.000000,0.069784,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.183514,0.000000,0.000000,0.091757,0.000000,0.000000,0.069784,0.091757,0.091757
1,0.093021,0.000000,0.000000,0.093021,0.000000,0.093021,0.000000,0.093021,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.093021,0.000000,0.093021,0.000000,0.070745,0.000000,0.000000
2,0.000000,0.065326,0.049682,0.000000,0.065326,0.000000,0.130652,0.000000,0.065326,0.065326,...,0.065326,0.000000,0.065326,0.000000,0.000000,0.000000,0.065326,0.000000,0.000000,0.000000


In [65]:
df["technology"]

0    0.209351
1    0.000000
2    0.298092
Name: technology, dtype: float64